**CHAT LOG**

LOP MODEL: https://chatgpt.com/share/68027ea6-a914-8004-a136-bf324053ffa1

GUI: https://chatgpt.com/share/6803733c-b028-8012-82d9-eee3812d619f

Maps Generation: Lecture Hands on Session Notes

Pickles: Lecture Hands on Session Notes

SELECT ALL TO RUN

**Imports and Setup**

In [4]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import folium
from tkinterweb import HtmlFrame
import webbrowser
import osmnx as ox
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import os
from PIL import Image, ImageTk
import requests
import io
import pickle


# Load attractions data from CSV
file_path = "attractions.csv"  
df = pd.read_csv(file_path)

# Convert CSV data into a dictionary for quick lookup
attraction_data = {row['name']: row for _, row in df.iterrows()}

**Functions**

In [6]:
# Clear all inputs
def clear_entries():
    start_time_entry.delete(0, tk.END)
    return_time_entry.delete(0, tk.END)
    for widget in attraction_frame.winfo_children():
        widget.destroy()
    selected_attractions.clear()

def minutes_to_hhmm(total_minutes):
    """Utility to convert 'total_minutes' into 'HH:MM' string."""
    h = int(total_minutes // 60)
    m = int(total_minutes % 60)
    return f"{h:02d}:{m:02d}"

def format_visit_time(minutes):
    """Convert visit duration (in minutes) into 'Xh Ym' format."""
    h = minutes // 60
    m = minutes % 60
    if h > 0 and m > 0:
        return f"{h}h {m}m"
    elif h > 0:
        return f"{h}h"
    else:
        return f"{m}m"

def download_image(image_url):
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes
        image_data = io.BytesIO(response.raw.read())
        return image_data
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")
        return None

def display_image(image_url, root, bg_color):
    image_data = download_image(image_url)
    if image_data:
        try:
            img = Image.open(image_data)
            img = img.resize((1250, 125), Image.LANCZOS)  # Resize the image
            photo = ImageTk.PhotoImage(img)
            label = tk.Label(root, image=photo, bg=bg_color)
            label.image = photo  # Keep a reference to avoid garbage collection
            label.pack()
        except Exception as e:
            print(f"Error processing image: {e}")    

def update_scroll_region(event):
    canvas.configure(scrollregion=canvas.bbox("all"))

    
# Handle attraction selection
def toggle_attraction(attraction):
    if attraction in selected_attractions:
        selected_attractions[attraction]['frame'].destroy()
        del selected_attractions[attraction]
    else:
        frame = tk.Frame(attraction_frame, bg="#e5e2db")
        frame.pack(fill="x", pady=2)

        tk.Label(frame, text=attraction, font=("Arial", 10), width=25, anchor="w", bg="#e5e2db").pack(side="left", padx=5)
        
        ranking_var = tk.StringVar(value="1")
        tk.Spinbox(frame, from_=1, to=10, textvariable=ranking_var, width=5).pack(side="left", padx=(25,0))
        
        hours_var = tk.StringVar(value="1")
        tk.Spinbox(frame, from_=0, to=12, textvariable=hours_var, width=3).pack(side="left", padx=(75,0))
        tk.Label(frame, text="H", bg="#e5e2db").pack(side="left")
        
        minutes_var = tk.StringVar(value="0")
        tk.Spinbox(frame, from_=0, to=59, increment=15, textvariable=minutes_var, width=3).pack(side="left", padx=5)
        tk.Label(frame, text="M", bg="#e5e2db").pack(side="left")
        
        #remove_btn = tk.Button(
            #frame,
            #text="Remove",
            #command=lambda f=frame, n=attraction_name: remove_attraction(f, n),
            #bg="#f44336",
            #fg="black"
        #)
        #remove_btn.pack(side="right")
        
        
        
        #ranking_entry = tk.Entry(frame, width=5)
        #ranking_entry.pack(side="left", padx=(25,0))

        #hours_entry = tk.Entry(frame, width=5)
        #hours_entry.pack(side="left", padx=(75,0))
        
        #tk.Label(frame, text="H", bg="#e5e2db").pack(side="left")

        #minutes_entry = tk.Entry(frame, width=5)
        #minutes_entry.pack(side="left", padx=5)

        #tk.Label(frame, text="M", bg="#e5e2db").pack(side="left")

        # Fetch attraction details from CSV (but don't display them)
        details = attraction_data.get(attraction, {})
        lat = details.get('latitude', None)
        long = details.get('longitude', None)
        open_time = details.get('opening', None)
        close_time = details.get('closing', None)

        selected_attractions[attraction] = {
            'frame': frame,
            'ranking': ranking_var,
            'hours': hours_var,
            'minutes': minutes_var,
            'latitude': lat,
            'longitude': long,
            'opening': open_time,
            'closing': close_time
        }

def update_layout(event=None):
    cols = 5
    
    # Clear existing buttons
    for widget in attraction_button_frame.winfo_children():
        widget.destroy()
    
    # Re-add buttons with updated layout
    for i, attraction in enumerate(attractions):
        row = i // cols  # Row index
        col = i % cols   # Column index
        
        tk.Button(attraction_button_frame, text=attraction, width=25, relief="ridge", bg="#d0cbbd",
                  command=lambda a=attraction: toggle_attraction(a)).grid(row=row, column=col, padx=5, pady=5)

def save_data():
    start_time = start_time_entry.get()
    return_time = return_time_entry.get()
    
    if not selected_attractions:
        messagebox.showerror("Error", "Please select at least one attraction!")
        return
    
    row_data = [start_time, return_time]
    attraction_columns = []

    for i, (attraction, entries) in enumerate(selected_attractions.items(), start=1):
        ranking = entries['ranking'].get()
        hours = entries['hours'].get()
        minutes = entries['minutes'].get()
        lat = entries['latitude']
        long = entries['longitude']
        open_time = entries['opening']
        close_time = entries['closing']

        if not (ranking and hours and minutes):
            messagebox.showerror("Error", f"Please fill in all fields for {attraction}!")
            return

        row_data.extend([attraction, lat, long, open_time, close_time, ranking, hours, minutes])

        attraction_columns.extend([
            f"attraction{i}",
            f"attraction{i}_lat",
            f"attraction{i}_long",
            f"attraction{i}_open",
            f"attraction{i}_close",
            f"attraction{i}_ranking",
            f"attraction{i}_hours",
            f"attraction{i}_minutes"
        ])

    df = pd.DataFrame([row_data], columns=["start_time", "return_time"] + attraction_columns)

    try:
        existing_df = pd.read_csv("itinerary.csv")
        updated_df = pd.concat([existing_df, df], ignore_index=True)
    except FileNotFoundError:
        updated_df = df

    updated_df.to_csv("itinerary.csv", index=False)
    
    # First, run the optimization to compute the route.
    optimize_itinerary_selective()
    
    # Then, display the itinerary and map.
    show_itinerary_popup()
    
    clear_entries()

def build_no_wait_itinerary(optimized_order, sol_X, nodes_data, distance_matrix, start_time_val):
    """
    Return a list of itinerary steps, each with: attraction, start_time, end_time, travel_time, visit_time.
    We forcibly set the schedule so that you leave as soon as possible from each node.
    """
    itinerary = []
    current_time = start_time_val  # Force the first node's "start time"
    avg_speed = 800  # match your solver's speed

    for idx in range(len(optimized_order)):
        node = optimized_order[idx]
        if idx == 0:
            # The first node is the hotel. Typically no "travel" to get here,
            # so start_time = user input, end_time = same (no visit at hotel).
            itinerary.append({
                "attraction": node,
                "start_time": current_time,
                "end_time": current_time,
                "travel_time": 0,
                "visit_time": 0
            })
        else:
            prev = optimized_order[idx - 1]
            dist = distance_matrix[(prev, node)]
            travel_minutes = dist / avg_speed

            # arrival time at node
            arrival_time = itinerary[-1]["end_time"] + travel_minutes
            # if we want to wait for open_time:
            arrival_time = max(arrival_time, nodes_data[node]['open'])

            visit_dur = nodes_data[node]['pref']
            departure_time = arrival_time + visit_dur

            itinerary.append({
                "attraction": node,
                "start_time": arrival_time,
                "end_time": departure_time,
                "travel_time": travel_minutes,
                "visit_time": visit_dur
            })

    return itinerary


**Itinerary Display and Map Generation**

In [8]:
# Global initializations (if not already defined)
optimized_order = []   # Final tour (list of node names in order)
selected_routes = []   # List of arcs used in the tour
nodes_global = {}      # Mapping for nodes (for mapping purposes)
final_itinerary_details = [] # Build final itinerary details row by row

# Fixed hotel: Marina Bay Sands (MBS)
MBS_NAME = "Marina Bay Sands"
MBS_LAT = 1.2838
MBS_LON = 103.8591
def format_time(delta_minutes):
    base_time = datetime(2025,1,1,9,0)# starting at 9AM
    new_time = base_time+timedelta(minutes=delta_minutes)
    return new_time.strftime("%H:%M")
    
# Global variables
optimized_order = []   
selected_routes = []   
nodes_global = {}      
final_itinerary_details = [] 

# Fixed hotel: Marina Bay Sands (MBS)
MBS_NAME = "Marina Bay Sands"
MBS_LAT = 1.2838
MBS_LON = 103.8591

def format_time(delta_minutes):
    base_time = datetime(2025, 1, 1, 9, 0)  
    new_time = base_time + timedelta(minutes=delta_minutes)
    return new_time.strftime("%H:%M")

def show_itinerary_popup():
    global selected_routes, optimized_order
    popup = tk.Toplevel(root)
    popup.title("Itinerary Plan")
    popup.geometry("1255x600")
    popup.configure(bg="#e5e2db")
    ROUTE_COLORS = ["blue", "green", "purple", "black", "brown", "gray"]
    
    
    # Frame for Image
    image_frame = tk.Frame(popup, bg="#e5e2db")
    image_frame.pack(pady=5)

    # Load image
    image_url = "https://www.marinabaysands.com/content/dam/mbsdam-assetshare/banner/brand-asset-skinny-banner.jpg"
    display_image(image_url, image_frame, "#e5e2db") # Pass background color
    
    
    
    header_label = tk.Label(popup, text="Optimized Itinerary", font=("Arial", 12, "bold"), bg="#e5e2db")
    header_label.pack(pady=5)
    
    columns = ("#", "Attraction", "Start Time", "End Time", "Travel Time", "Visit Time")
    tree = ttk.Treeview(popup, columns=columns, show="headings")
    
    for col in columns:
        tree.heading(col, text=col)

    tree.column("#", width=30, anchor="center")
    tree.column("Attraction", width=200, anchor="w")
    tree.column("Start Time", width=100, anchor="center")
    tree.column("End Time", width=100, anchor="center")
    tree.column("Travel Time", width=100, anchor="center")
    tree.column("Visit Time", width=100, anchor="center")
    
    tree.pack(pady=10, padx=10)

    # Populate itinerary table
    for i, step in enumerate(final_itinerary_details, start=1):
        tree.insert("", "end", values=(i, step["attraction"], step["start_time"], step["end_time"], step["travel_time"], step["visit_time"]))

    locs = [(MBS_NAME, MBS_LAT, MBS_LON)]
    
    # Build attraction list
    for attraction in optimized_order:
        if attraction == MBS_NAME:
            continue
        details = attraction_data.get(attraction, {})
        try:
            lat = float(details.get('latitude'))
            lon = float(details.get('longitude'))
            locs.append((attraction, lat, lon))
        except Exception as e:
            print(f"Skipping {attraction}: {e}")

    # Generate map
    map_file = "itinerary_map.html"
    m = folium.Map(location=[MBS_LAT, MBS_LON], zoom_start=12)

    # Add distinct marker for starting location
    folium.Marker(
        location=[MBS_LAT, MBS_LON],
        popup=f"<b>{MBS_NAME} (Start)</b>",
        icon=folium.Icon(color="red", icon="star", prefix="fa")  
    ).add_to(m)

    # Assign colors to each location (excluding MBS)
    location_colors = {}
    for idx, (attraction, lat, lon) in enumerate(locs[1:], start=1):  
        color = ROUTE_COLORS[idx % len(ROUTE_COLORS)]
        location_colors[attraction] = color

    # Add markers with matched colors
    for idx, (attraction, lat, lon) in enumerate(locs[1:], start=1):
        color = location_colors[attraction]
        folium.Marker(
            location=[lat, lon],
            popup=f"<b>{idx}. {attraction}</b>",
            icon=folium.DivIcon(
                icon_size=(30, 30),
                icon_anchor=(15, 15),
                html=f'<div style="background-color:{color};color:white;border-radius:50%;border:2px solid black;width:30px;height:30px;line-height:30px;text-align:center;font-weight:bold;">{idx}</div>'
            )
        ).add_to(m)
    
    # Build routes using shortest paths with colors matching the locations
    graph = ox.graph_from_place("Singapore", network_type='drive')
    nodes_map = {MBS_NAME: ox.distance.nearest_nodes(graph, MBS_LON, MBS_LAT)}

    for attraction, lat, lon in locs[1:]:
        nodes_map[attraction] = ox.distance.nearest_nodes(graph, lon, lat)

    if not optimized_order:
        print("No optimized order available to draw.")
    else:
        for i in range(len(optimized_order) - 1):
            loc1, loc2 = optimized_order[i], optimized_order[i+1]
            try:
                path = nx.shortest_path(graph, nodes_map[loc1], nodes_map[loc2], weight='length')
                path_coords = [(graph.nodes[n]['y'], graph.nodes[n]['x']) for n in path]
                color = location_colors.get(loc2, "black")  # Match route color with location
                
                # ADD ROUTE COLOR MATCHING MARKER
                folium.PolyLine(
                    locations=path_coords,
                    color=color,
                    weight=3,
                    opacity=0.8,
                    tooltip=f"Route {i+1}: {loc1} → {loc2}"
                ).add_to(m)

            except Exception as e:
                print(f"Could not add route from {loc1} to {loc2}: {e}")

    m.save(map_file)
    print("Map saved to", map_file)

    # Button to open the map
    def open_map():
        map_path = os.path.abspath(map_file)
        webbrowser.open("file://" + map_path)

    tk.Button(popup, text="View Map", command=open_map, bg="#2196F3", fg="black", font=("Arial", 10, "bold")).pack(pady=10)
    tk.Button(popup, text="Close", command=popup.destroy, font=("Arial", 10, "bold"), bg="#f44336", fg="black").pack(pady=10)

**LOP Model**

In [10]:
def optimize_itinerary_selective():
    global optimized_order, nodes_global, selected_routes

    if not selected_attractions:
        messagebox.showerror("Error", "Please select at least one attraction!")
        return

    # --- Overall Time Window ---
    try:
        start_time_val = int(start_time_entry.get().split(":")[0]) * 60 + int(start_time_entry.get().split(":")[1])
        return_time_val = int(return_time_entry.get().split(":")[0]) * 60 + int(return_time_entry.get().split(":")[1])
    except Exception as e:
        messagebox.showerror("Error", "Invalid start/return time format. Please use HH:MM.")
        return

    # --- Hotel (Fixed Starting/Ending Point) ---
    hotel = "Marina Bay Sands"
    hotel_lat = 1.2838
    hotel_lon = 103.8591

    # --- Build Nodes Data ---
    nodes_data = {}
    nodes_data[hotel] = {
        'lat': hotel_lat,
        'lon': hotel_lon,
        'pref': 0,
        'open': start_time_val,
        'close': return_time_val,
        'rank': 0
    }
    for attraction, entries in selected_attractions.items():
        try:
            lat = float(entries['latitude'])
            lon = float(entries['longitude'])
        except Exception as e:
            messagebox.showerror("Error", f"Invalid coordinates for {attraction}: {e}")
            return
        try:
            pref = int(entries['hours'].get()) * 60 + int(entries['minutes'].get())
            open_val = int(entries['opening'].split(":")[0]) * 60 + int(entries['opening'].split(":")[1])
            close_val = int(entries['closing'].split(":")[0]) * 60 + int(entries['closing'].split(":")[1])
            rank_val = int(entries['ranking'].get())
        except Exception as e:
            messagebox.showerror("Error", f"Invalid time or ranking for {attraction}: {e}")
            return
        nodes_data[attraction] = {
            'lat': lat,
            'lon': lon,
            'pref': pref,
            'open': open_val,
            'close': close_val,
            'rank': rank_val
        }
    print("Nodes data for optimization:")
    for n in nodes_data:
        print(n, nodes_data[n])
    
    # --- Compute Scores ---
    ranks = [nodes_data[n]['rank'] for n in nodes_data if n != hotel]
    max_rank = max(ranks) if ranks else 1
    score = {n: (max_rank - nodes_data[n]['rank'] + 1) if n != hotel else 0 for n in nodes_data}
    print("Score for each node:", score)
    
    # --- Load Road Network and Build Distance Matrix ---
    G = ox.graph_from_place("Singapore", network_type='drive')
    nearest_nodes = {n: ox.distance.nearest_nodes(G, X=nodes_data[n]['lon'], Y=nodes_data[n]['lat']) for n in nodes_data}

    # Setup for caching
    distance_pickle_path = "distance_matrix.pkl"
    node_keys = tuple(sorted(nodes_data.keys()))
    pickle_key = f"matrix_{hash(node_keys)}"

    # Load cache if it exists
    if os.path.exists(distance_pickle_path):
        with open(distance_pickle_path, "rb") as f:
            all_cached_matrices = pickle.load(f)
    else:
        all_cached_matrices = {}

    # Load distance matrix from cache or compute it
    if pickle_key in all_cached_matrices:
        distance_matrix = all_cached_matrices[pickle_key]
    else:
        distance_matrix = {}
        for i in nodes_data:
            for j in nodes_data:
                if i != j:
                    try:
                        d = nx.shortest_path_length(G, nearest_nodes[i], nearest_nodes[j], weight='length')
                        distance_matrix[(i, j)] = d
                    except Exception as e:
                        print(f"Error computing distance from {i} to {j}: {e}")
                        distance_matrix[(i, j)] = float("inf")
        # Save to cache
        all_cached_matrices[pickle_key] = distance_matrix
        with open(distance_pickle_path, "wb") as f:
            pickle.dump(all_cached_matrices, f)

    # Print result
    print("Computed distance_matrix:")
    for arc in distance_matrix:
        print(f"{arc}: {distance_matrix[arc]}")
    
    nodes_global = nearest_nodes

    # --- Gurobi Model Setup ---
    model = gp.Model("selective_itinerary")
    model.Params.OutputFlag = 0

    # Decision variables
    y = model.addVars(nodes_data.keys(), vtype=GRB.BINARY, name="y")
    y[hotel].LB = 1  # Hotel must be visited.
    X = model.addVars(distance_matrix.keys(), vtype=GRB.BINARY, name="X")
    S = model.addVars(nodes_data.keys(), vtype=GRB.CONTINUOUS, name="S")
    
    # --- Path Continuation Constraints ---
    M_val = 10000  # Big-M constant
    avg_speed = 800  # Average travel speed in meters per minute
    
    # (1) Start from hotel
    model.addConstr(gp.quicksum(X[hotel, j] for j in nodes_data if j != hotel) == 1, "HotelOut")
    
    # (2) End at hotel
    model.addConstr(gp.quicksum(X[i, hotel] for i in nodes_data if i != hotel) == 1, "HotelIn")
    
    # (3) If node i is visited, exactly one path leaves attraction i
    for i in nodes_data:
        if i == hotel:
            continue
        model.addConstr(gp.quicksum(X[i, j] for j in nodes_data if j != i) == y[i], f"Out_{i}")
        
    # (4) If node i is visited, exactly one path enters attraction i    
        model.addConstr(gp.quicksum(X[j, i] for j in nodes_data if j != i) == y[i], f"In_{i}")
    
    for (i, j) in distance_matrix:
    # (5) If path from i to j is used, attraction i must be visited
        model.addConstr(X[i, j] <= y[i], f"Link_{i}_{j}")
        
    # (6) If path from i to j is used, attraction j must be visited
        model.addConstr(X[i, j] <= y[j], f"Link2_{i}_{j}")
    
    # --- Return Time Constraints ---
    for (i, j) in distance_matrix:
        travel_time = distance_matrix[(i, j)] / avg_speed
        if j == hotel:
            model.addConstr(S[i] + nodes_data[i]['pref'] + travel_time <= return_time_val + M_val*(1 - X[i, j]),
                            f"ReturnTime_{i}")
    # --- Time Ordering Constraints ---        
        else:
            model.addConstr(S[j] >= S[i] + nodes_data[i]['pref'] + travel_time - M_val*(1 - X[i, j]),
                            f"TimeOrder_{i}_{j}")
    
    # --- Operating Hours Constraints ---
    
    # (9) S_i ≥ O_i - M(1 - y_i)
        # If the attraction is visited (y_i = 1), ensure that S_i (start time) is after the opening time
    for i in nodes_data:
        model.addConstr(S[i] >= nodes_data[i]['open'] - M_val*(1 - y[i]), f"Open_{i}")
        
    # (10) S_i + p_i ≤ C_i + M(1 - y_i)
        # If the attraction is visited (y_i = 1), ensure that the visit ends before the closing time (S_i + p_i)    
        model.addConstr(S[i] + nodes_data[i]['pref'] <= nodes_data[i]['close'] + M_val*(1 - y[i]),
                        f"Close_{i}")
    
    # --- Ranking Constraints ---
    def compute_bonus(r):
        ranks = [nodes_data[n]['rank'] for n in nodes_data if n != hotel]
        max_rank = max(ranks) if ranks else 1
        return (max_rank - r + 1) * 100  # higher rank → higher bonus

    bonus = {n: compute_bonus(nodes_data[n]['rank']) if n != hotel else 0 for n in nodes_data}


    print("Revised bonus for each node:", bonus)
    
    # Set objective function: Maximize bonus - penalty for travel distance
    lambda_penalty = 0.01  # Penalty weight for travel distance.
    model.setObjective(
        gp.quicksum(bonus[i] * y[i] for i in nodes_data if i != hotel)
        - lambda_penalty * gp.quicksum(distance_matrix[i, j] * X[i, j] for (i, j) in distance_matrix),
        GRB.MAXIMIZE
    )
    
    model.optimize()
    if model.status == GRB.OPTIMAL:
        sol_y = model.getAttr("x", y)
        sol_X = model.getAttr("x", X)
        sol_S = model.getAttr("x", S)  # <--- Start times

        selected_routes = [arc for arc, val in sol_X.items() if val > 0.5]
        
        # --- Reconstruct the Tour ---
        current = hotel
        order = [current]
        while True:
            next_arc = [arc for arc in selected_routes if arc[0] == current]
            if not next_arc:
                break
            next_node = next_arc[0][1]
            order.append(next_node)
            current = next_node
            if current == hotel:
                break
        optimized_order = order
    
    # Build a forced no-wait schedule, ignoring the solver's S[node].
    global final_itinerary_details
    final_list = build_no_wait_itinerary(optimized_order, sol_X, nodes_data, distance_matrix, start_time_val)

    # Convert final_list times to strings, store in final_itinerary_details
    final_itinerary_details = []
    for step in final_list:
        start_str = minutes_to_hhmm(step["start_time"])
        end_str = minutes_to_hhmm(step["end_time"])
        travel_str = format_visit_time(int(step["travel_time"]))
        visit_str = format_visit_time(int(step["visit_time"]))
        final_itinerary_details.append({
            "attraction": step["attraction"],
            "start_time": start_str,
            "end_time": end_str,
            "travel_time": travel_str,
            "visit_time": visit_str
        })

GUI code

In [ ]:
# GUI
root = tk.Tk()
root.title("WanderWise")
root.geometry("1380x800")
root.configure(bg="#e5e2db")

# Canvas and Scrollbar
canvas = tk.Canvas(root, bg="#e5e2db", scrollregion=(0, 0, 1250, 600))  
canvas.pack(side="left", fill="both", expand=True)

scrollbar = ttk.Scrollbar(root, orient=tk.VERTICAL, command=canvas.yview)
scrollbar.pack(side="right", fill="y")

canvas.configure(yscrollcommand=scrollbar.set)

# Frame for Scrollable Content
scrollable_frame = tk.Frame(canvas, bg="#e5e2db")
canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")


# Load image
image_url = "https://www.marinabaysands.com/content/dam/mbsdam-assetshare/banner/brand-asset-skinny-banner.jpg" 
display_image(image_url, scrollable_frame, "#e5e2db")


# Title
tk.Label(scrollable_frame, text="Travel Itinerary Planner", font=("Arial", 16, "bold"), bg="#e5e2db").pack(pady=10)



# Input Frame
frame = tk.Frame(scrollable_frame, bg="#e5e2db", padx=10, pady=10, relief=tk.RIDGE, borderwidth=2)
frame.pack(pady=10, padx=10, fill="x")

# Time Inputs
tk.Label(frame, text="Start Time from Hotel (24-hour clock, e.g. 23:59):", bg="#e5e2db").pack(anchor="w")
start_time_entry = tk.Entry(frame, width=20)
start_time_entry.pack(pady=2)

tk.Label(frame, text="Return Time to Hotel (24-hour clock, e.g., 23:59):", bg="#e5e2db").pack(anchor="w")
return_time_entry = tk.Entry(frame, width=20)
return_time_entry.pack(pady=2)

# Attractions Section
tk.Label(scrollable_frame, text="Select Attractions:", font=("Arial", 12, "bold"), bg="#e5e2db").pack(anchor="w", padx=20, pady=5)
attraction_button_frame = tk.Frame(scrollable_frame, bg="#e5e2db")
attraction_button_frame.pack(pady=5)

attractions = list(attraction_data.keys())

selected_attractions = {}

# Headers for Selected Attractions
header_frame = tk.Frame(scrollable_frame, bg="#e5e2db")
header_frame.pack(fill="x", padx=20)

tk.Label(header_frame, text="Attraction", font=("Arial", 10, "bold"), width=24, bg="#e5e2db", anchor="w").pack(side="left", padx=5)
tk.Label(header_frame, text="Ranking", font=("Arial", 10, "bold"), width=13, bg="#e5e2db").pack(side="left", padx=5)
tk.Label(header_frame, text="Preferred Time Spent", font=("Arial", 10, "bold"), width=24, bg="#e5e2db").pack(side="left", padx=5)

# Frame for Selected Attractions
attraction_frame = tk.Frame(scrollable_frame, bg="#e5e2db")
attraction_frame.pack(fill="x", padx=20, pady=5)

# Buttons
button_frame = tk.Frame(scrollable_frame, bg="#e5e2db")
button_frame.pack(pady=10)

tk.Button(button_frame, text="Submit", command=save_data, font=("Arial", 10, "bold"), bg="#4CAF50", fg="black", padx=10, pady=5).pack(side="left", padx=5)
tk.Button(button_frame, text="Clear", command=clear_entries, font=("Arial", 10, "bold"), bg="#f44336", fg="black", padx=10, pady=5).pack(side="right", padx=5)


scrollable_frame.bind("<Configure>", update_scroll_region)

update_layout()
root.mainloop()

Nodes data for optimization:
Marina Bay Sands {'lat': 1.2838, 'lon': 103.8591, 'pref': 0, 'open': 540, 'close': 900, 'rank': 0}
Singapore Zoo {'lat': 1.4043, 'lon': 103.793, 'pref': 60, 'open': 510, 'close': 1080, 'rank': 1}
Singapore Flyer {'lat': 1.2893, 'lon': 103.8631, 'pref': 60, 'open': 600, 'close': 1320, 'rank': 1}
Score for each node: {'Marina Bay Sands': 0, 'Singapore Zoo': 1, 'Singapore Flyer': 1}
Computed distance_matrix:
('Marina Bay Sands', 'Singapore Zoo'): 22059.167710538994
('Marina Bay Sands', 'Singapore Flyer'): 1267.4769004989441
('Singapore Zoo', 'Marina Bay Sands'): 22271.6661526833
('Singapore Zoo', 'Singapore Flyer'): 21384.943579329818
('Singapore Flyer', 'Marina Bay Sands'): 1234.9396555384324
('Singapore Flyer', 'Singapore Zoo'): 21605.7439548291
Restricted license - for non-production use only - expires 2026-11-23
Revised bonus for each node: {'Marina Bay Sands': 0, 'Singapore Zoo': 100, 'Singapore Flyer': 100}
Map saved to itinerary_map.html
